In [1]:
# imports

import os
from dotenv import load_dotenv

from openai import OpenAI
import anthropic
import google.generativeai

from IPython.display import Markdown, display, update_display

# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyC_


In [2]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

# OPENAI
https://platform.openai.com/docs/api-reference/project-rate-limits/object

In [10]:
MODEL_GPT = "gpt-4o-mini"

In [ ]:
# chat completion in openai
openai = OpenAI()
completion = openai.chat.completions.create(model=MODEL_GPT, messages=prompts)
print(completion.choices[0].message.content)

Why do data scientists prefer dark mode?  

Because light attracts bugs! 🐞✨


In [9]:
# GPT-4o-mini
# Temperature setting controls creativity
# chat completion in openai

# max_tokens:
# The maximum number of tokens that can be generated in the completion.

# The token count of your prompt plus max_tokens cannot exceed the model's context length.

openai = OpenAI()
completion = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=prompts,
    temperature=0.7,
    max_tokens=None
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because she found him too mean!


In [12]:
# Streaming with Markdown
stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt}
      ],
        stream = True
)
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

Why do data scientists love nature?

Because it has the best "trees" for their "random forests!" 🌲😄

In [ ]:
# Streaming with print
stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt}
      ],
        stream = True
)
for chunk in stream:
    print(chunk.choices[0].delta.content, end="")

Why did the data scientist bring a ladder to work? 

Because they wanted to reach the next level of understanding!None

In [ ]:
# getting json as the output. 
import json
system_message = "You are a math tutor that solves simple and complex math problems and returns the answer in json format" # you can spcify how you want the output
user_prompt = "What is the sum of all integers from 1 to 100?" 

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]
openai = OpenAI()
completion = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=prompts,
    temperature=0.0,
    max_tokens=None,
    response_format={"type": "json_object"} # if this is not set, the output would contain text!
)

result =completion.choices[0].message.content
print(result)

json.loads(result)

{"sum":5050}


{'sum': 5050}

In [23]:
# getting json as the output. 
import json
system_message = "You are a math tutor that solves simple and complex math problems and returns the answer in json format with the following fields:\
    operands(a list containing the numbers used), result" # you can spcify how you want the output
user_prompt = "What is the sum of all integers from 1 to 100?" 

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]
openai = OpenAI()
completion = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=prompts,
    temperature=0.0,
    max_tokens=None,
    response_format={"type": "json_object"} # if this is not set, the output would contain text!
)

result =completion.choices[0].message.content
print(result)

json.loads(result)

{
  "operands": [1, 100],
  "result": 5050
}


{'operands': [1, 100], 'result': 5050}

## Chat

# Anthropic

keep in mind that in anthropic, the system prompt is given in the function call.

In [ ]:
# chat completion in anthropic
claude = anthropic.Anthropic()
message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Sure, here's a light-hearted joke for data scientists:

Why do data scientists prefer dark mode?

Because light attracts bugs!

This joke plays on the dual meaning of "bugs" - both as insects attracted to light and as errors in code that data scientists often have to debug. It's a fun little pun that combines a common preference among programmers (dark mode) with a data science-related concept.


In [29]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)
with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Here's a light-hearted joke for Data Scientists:

Why did the data scientist break up with their significant other?

There was just too much multicollinearity in their relationship!

(Explanation for non-data scientists: Multicollinearity is a statistical concept where two or more variables in a model are highly correlated, making it difficult to determine their individual effects. In relationships, it might suggest that the couple was too similar or their lives were too intertwined, leading to complications!)

# Gemini

In [3]:
# The API for Gemini has a slightly different structure.
google.generativeai.configure() # if you don't run this, you will get an error
gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the data scientist sad?  Because they didn't get the *array* they wanted!



In [5]:
stream_gemini_results('Hi')

<generator object stream_gemini_results at 0x10ac27640>